# OWIDplusLIVE

Das folgende Jupyter-Notebook dokumentiert die API von OWIDplusLIVE (www.owid.de/plus/live-2021)


## Dependencies

Folgende Dependencies müssen einmalig nach dem Starten des Notebooks importiert werden,


In [18]:
import requests
import json

## Statische Daten u. Funktionen

Folgende Variablen/Daten und Funktionen werden zentral im gesamten Notebook verwendet und müssen einmal nach dem Start des Notebooks ausgeführt werden:


In [17]:
baseUrl = "https://www.owid.de/plus/live-2021/api"
layer = {"Wortform": 0, "Lemma": 1, "POS": 2}

def search(query):
    url = f"{baseUrl}/find"
    payload = json.dumps(query)
    headers = {
        'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    return response.json()

def granulation_by_date(data):
    return data

def granulation_by_month(data):
    return data[:7]

def granulation_by_quarter(data):
    return data[:4] + "-Q" + str((int(data[5:7]) - 1) // 3 + 1)

def granulation_by_year(data):
    return data[:4]

def safeGetVal(data, key):
    if key in data:
        return data[key]
    return 0

def moving_average(normGramItem_Granulated, data_Granulated, n):
    if(n < 2):
        return data_Granulated
    
    res = {}
    dates = list(normGramItem_Granulated.keys())
    for i in range(len(dates) - n + 1):
        select = dates[i:i+n+1] if n%2 == 0 else dates[i:i+n]        
        first = safeGetVal(data_Granulated, select[0])
        center = select[len(select) // 2]
        last = safeGetVal(data_Granulated, select[-1])
        select = select[1:-1]
        
        x = 0.0
        if n % 2 == 0:
            x += first / 2
            x += last / 2
        else:
            x += first
            x += last
        
        for j in select:
            x += safeGetVal(data_Granulated, j)
        
        res[center] = x / (n + 1 if n % 2 == 0 else n)
    return res

### Notizen

Die Funktion search (siehe oben) erlaubt die Suche nach


In [ ]:
#def searchN1(layer1, token1):
#    query = {
#        "N": 1,
#        "Items": [
#            {
#                "layer": layer1,
#                "token": token1,
#                "position": 0
#            }
#        ]
#    }    
#    return search(query)
#
#def searchN2(layer1, token1, layer2, token2):
#    query = {
#        "N": 2,
#        "Items": [
#            {
#                "layer": layer1,
#                "token": token1,
#                "position": 0
#            },
#            {
#                "layer": layer2,
#                "token": token2,
#                "position": 1
#            }
#        ]
#    }    
#    return search(query)
#
#def searchN3(layer1, token1, layer2, token2, layer3, token3):
#    query = {
#        "N": 3,
#        "Items": [
#            {
#                "layer": layer1,
#                "token": token1,
#                "position": 0
#            },
#            {
#                "layer": layer2,
#                "token": token2,
#                "position": 1
#            },
#            {
#                "layer": layer3,
#                "token": token3,
#                "position": 2
#            }
#        ]
#    }    
#    return search(query)

## Normadaten

Einmalig nach dem Start müssen die aktuellen Normdaten bezogen werden - diese werden zur Berechnung der relativen Frequenz verwendet.


In [16]:
url = f"{baseUrl}/norm"

payload = {}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)
response = response.json()

def normClean(response, idx):
    res = response[idx]
    res = {key.split("T")[0]: res[key] for key in res}
    res = dict(sorted(res.items()))
    return res

normGram = {}
normGram[1] = normClean(response, 0)
normGram[2] = normClean(response, 1)
normGram[3] = normClean(response, 2)

print("TOTAL:")
print(f"normGram 1: {sum([normGram[1][x] for x in normGram[1]])}")
print(f"normGram 2: {sum([normGram[2][x] for x in normGram[2]])}")
print(f"normGram 3: {sum([normGram[3][x] for x in normGram[3]])}")

TOTAL:
normGram 1: 142537047
normGram 2: 128811391
normGram 3: 115111122


##
